# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df = pd.read_csv('preprocessed_descriptions.csv')
df

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sister in law made these for us at a family...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...
29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought...
29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,zydeco soup,this is a delicious soup that i originally fou...


In [53]:
from nltk.tokenize import word_tokenize

words1 = set() #мнодество удаляет повторы
for i in df['preprocessed_descriptions'].values:
    words1.update(word_tokenize(str(i).lower())) 
#используем update, а не add, т к добавляем не один элемент, а список!
a = list(words1) # в массив
words = set()
for x in a:
    if x.isalpha(): # проверка на слово 
        words.add(x)
len(words)

23037

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [54]:
import random
random_pairs = random.sample(words, k=10)  # Выбираем 10 случайных слов из unique_words
for i in range(0, len(random_pairs), 2):
    word1 = random_pairs[i]
    word2 = random_pairs[i + 1]
    edit_distance = nltk.edit_distance(word1, word2) # расстояние Левенштейна
    print(f"Расстояние между '{word1}' и '{word2}': {edit_distance}")

Расстояние между 'fulls' и 'wait': 5
Расстояние между 'engaged' и 'pommeau': 7
Расстояние между 'tamed' и 'yuuuuuuuuummmmm': 14
Расстояние между 'tavern' и 'hasty': 5
Расстояние между 'paint' и 'dariole': 5


C:\Users\sibir\AppData\Local\Temp\ipykernel_10580\531333538.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_pairs = random.sample(words, k=10)  # Выбираем 10 случайных слов из unique_words


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [55]:
def find(word, k):
    dis = {} # Словарь для хранения расстояний между словами и заданным словом
    for i in words:
        r = nltk.edit_distance(word, i) # Вычисление расстояния Левенштейна
        dis[i] = r
    #print(dis)
    # ключ - слово, значение - расстояние Левеншиейна r
    # через get достаются значения из словаря dis, key - по какому принципу мы сортируем словарь
    sor = sorted(dis, key=dis.get)[:k]
    # Сортировка слов по расстоянию и выбор k ближайших слов
    return f"Ближайшие {k} слова к '{word}': {sor}"

In [56]:
print(find('parm', 4))

Ближайшие 4 слова к 'parm': ['parm', 'palm', 'farm', 'para']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [57]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sibir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sibir\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [58]:
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
# в таблицу можно добавлять только массивы
words = list(words)
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

dt = pd.DataFrame({'word': words, 'stemmed_word': [stemmer.stem(i) for i in words], 
                      'normalized_word': [lemmatizer.lemmatize(i) for i in words]})
dt.set_index('word')




,stemmed_word,normalized_word
word,,
whoever,whoever,whoever
serves,serv,serf
parmenter,parment,parmenter
tennesse,tenness,tennesse
overwhelmingly,overwhelm,overwhelmingly
...,...,...
ferrero,ferrero,ferrero
ususally,usus,ususally
wobbly,wobbl,wobbly


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [59]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sibir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
from collections import Counter
#словарь: слово + кол-во вхождений
counter = Counter()

for i in df['preprocessed_descriptions']:
    counter.update(word_tokenize(str(i).lower()))
#print(counter)
n_words_old = sum(counter.values())
n_words_old

1104518

In [61]:
counter.most_common(10)

[('the', 40413),
 ('a', 35131),
 ('and', 30585),
 ('i', 27945),
 ('this', 27181),
 ('to', 23598),
 ('it', 23300),
 ('is', 20306),
 ('of', 18405),
 ('for', 16023)]

In [62]:
from nltk.corpus import stopwords 

en_stopwords = stopwords.words('english')

keys = list(counter.keys())
for i in keys:
    if i in en_stopwords:
        counter.pop(i)
print(str(100 - sum(counter.values()) / n_words_old * 100) + '%') # достаем значения

47.09755748661407%


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
sample = df.sample(5)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(sample.preprocessed_descriptions.values).toarray() 
#приводим в векторый вид, записываем векорный вид в матрицу

vectors[0]

array([0.22311203, 0.        , 0.22311203, 0.        , 0.44826218,
       0.22311203, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.22311203, 0.        , 0.        ,
       0.        , 0.        , 0.22311203, 0.        , 0.        ,
       0.22311203, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.14942073, 0.        , 0.        ,
       0.22311203, 0.        , 0.        , 0.        , 0.18000538,
       0.18000538, 0.        , 0.        , 0.22311203, 0.        ,
       0.        , 0.22311203, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.18000538,
       0.        , 0.        , 0.        , 0.        , 0.22311203,
       0.        , 0.        , 0.22311203, 0.        , 0.22311

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [11]:
import pandas as pd
from scipy.spatial.distance import cosine

matrix = pd.DataFrame(index=range(5), columns=range(5) # пустой DataFrame

for i in range(5):
    for j in range(i, 5):
        similarity = cosine(vectors[i], vectors[j])
        matrix.loc[i, j] = similarity
        matrix.loc[j, i] = similarity


matrix.columns = ['Рецепт ' + str(i+1) for i in range(5)]
matrix.index = ['Рецепт ' + str(i+1) for i in range(5)]

matrix


,Рецепт 1,Рецепт 2,Рецепт 3,Рецепт 4,Рецепт 5
Рецепт 1,0,0.966728,0.874754,1.0,0.876163
Рецепт 2,0.966728,0,0.94748,1.0,0.862501
Рецепт 3,0.874754,0.94748,0,0.937014,0.733862
Рецепт 4,1.0,1.0,0.937014,0,0.867126
Рецепт 5,0.876163,0.862501,0.733862,0.867126,0


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

Чем меньше значение, тем более схожи слова(т е 0 означает 100 % совпадение слов)